# Mediapipe
* Mediapipe는 구글에서 개발한 오픈 소스 플랫폼 프레임워크으로 얼굴인식, 포즈, 객체감지, 모션트레킹 등 다양한 형태의 기능과 모델을 제공합니다.
* Mediapipe 사이트 > Solutions > See Demos에서 다양한 예제를 확인할 수 있습니다.
* Mediapipe는 현재 파이썬 3.8 ~ 3.11 버전에서 작동합니다.
    * https://developers.google.com/mediapipe/solutions/setup_python
    * 나중에 사용할 수 있는 파이썬 버전은 바뀔 수 있습니다.
* ```pip install mediapipe```로 설치합니다. 
    * user 관련 문제로 설치가 잘 안 되면 ```pip install --user mediapipe```로 설치합니다.

## mediapipe 사용해보기
* 손가락을 인식하는 프로그램을 실행해보겠습니다.

In [ ]:
import cv2
import mediapipe as mp

mp_drawing = mp.solutions.drawing_utils
mp_hands = mp.solutions.hands

capture = cv2.VideoCapture(0)

with mp_hands.Hands(
    max_num_hands=1,
    min_detection_confidence=0.5,
    min_tracking_confidence=0.5
    ) as hands:

    while capture.isOpened():
        ret, frame = capture.read()
        if not ret:
            continue
        frame = cv2.cvtColor(cv2.flip(frame, 1), cv2.COLOR_BGR2RGB)
        results = hands.process(frame)

        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)
        if results.multi_hand_landmarks:
            for hand_landmarks in results.multi_hand_landmarks:                            
                mp_drawing.draw_landmarks(frame, hand_landmarks, mp_hands.HAND_CONNECTIONS)
        cv2.imshow("Finger", frame)
        if cv2.waitKey(1) == 27:
            break
        
capture.release()
cv2.destroyAllWindows()

## 얼굴 인식하기
* mediapipe 사이트에 다양한 예제 코드가 있습니다.
* 얼굴을 촬영해서 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 위치를 확인합니다.

In [ ]:
import cv2
import mediapipe as mp

capture = cv2.VideoCapture(0)
                       
# 얼굴을 찾고, 찾은 얼굴에 표시를 해주기 위한 변수를 정의합니다.
# 얼굴 검출을 위해서 face_detection 모듈을 사용합니다.
mp_face_detection = mp.solutions.face_detection 
# 얼굴의 특징을 그리기 위해서 drawing_utils 모듈을 사용합니다.
mp_drawing = mp.solutions.drawing_utils 

# mediapipe 사용방법에 맞게 설정합니다.
# model_selection = 0은 카메라와 가까운 것을 인식합니다.
# model_selection = 1은 카메라와 먼 것을 인식합니다.
# min_detection_confidence=0.7는 70% 확신한다면 얼굴로 본다는 뜻입니다.
with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret: # 읽지 못했다면 
            continue             
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame)
       
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)        
        if results.detections:
            # 6개 특징 
            # 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 위치가 점을 표시됩니다.
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)  
                #print(detection)
        cv2.imshow('MediaPipe Face Detection', frame)
        
        if cv2.waitKey(1) == 27:
            break
            
capture.release()
cv2.destroyAllWindows()

* print(detection)으로 어떤 내용인지 확인합니다.
* 촬영을 하고 얼굴을 인식하면 조금 있다가 ESC키를 누릅니다.
* 6개의 relative_keypoints가 있습니다.
    * 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 좌푯값입니다.
    * 이 좌푯값을 사용해서 얼굴 필터를 만듭니다.

In [ ]:
import cv2
import mediapipe as mp

capture = cv2.VideoCapture(0)
                       
mp_face_detection = mp.solutions.face_detection 
mp_drawing = mp.solutions.drawing_utils 

with mp_face_detection.FaceDetection(model_selection=0, min_detection_confidence=0.7) as face_detection:
    while capture.isOpened():
        ret, frame = capture.read()
        if not ret: 
            continue             
        frame.flags.writeable = False
        frame = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        results = face_detection.process(frame)
       
        frame.flags.writeable = True
        frame = cv2.cvtColor(frame, cv2.COLOR_RGB2BGR)        
        if results.detections:
            # 6개 특징 
            # 오른쪽 눈, 왼쪽 눈, 코 끝부분, 입 중심, 오른쪽 귀, 왼쪽 귀 위치가 점을 표시됩니다.
            for detection in results.detections:
                mp_drawing.draw_detection(frame, detection)  
                # 위치를 나타내는 것을 keypoints 리스트에 저장합니다.
                keypoints = detection.location_data.relative_keypoints
                # keypoints[0]은 오른쪽 눈입니다.
                # keypoints[0].x는 x좌표이고 keypoints[0].y는 y좌표입니다.
                nose = keypoints[2]
                # frame의 크기로 계산합니다.
                # frame.shape은 높이, 너비, 채널 값입니다.
                h, w, c = frame.shape  
                # 튜플로 다시 저장합니다.
                nose = (int(nose.x * w), int(nose.y * h))
                # 원을 그립니다.
                cv2.circle(frame, nose, 50, (0,0,255), 10, cv2.LINE_AA)
        cv2.imshow('MediaPipe Face Detection', frame)
        
        if cv2.waitKey(1) == 27:
            break
            
capture.release()
cv2.destroyAllWindows()

# 얼굴 필터 만들기
* 양쪽 눈에 그림을 그려주는 프로그램을 만들어보세요.
    * 사각형이나 원을 그려줍니다.